# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [3]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
    """
)

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are healthy in moderation and as part of a balanced diet.
    
    Key benefits:
    - Good source of potassium, vitamin B6, vitamin C, and fiber
    - Provide quick energy from natural sugars and starch
    - Contain resistant starch (especially when less ripe) that can help gut health
    
    Considerations:
    - Natural sugars mean they’re higher in carbs than some fruits; portion if you’re watching sugar or calories
    - People with kidney disease may need to monitor potassium intake
    - Pair with protein or fat for lasting fullness
    
    One medium banana (about 118 g) has roughly 105 calories.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [6]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are healthy as part of a balanced diet.

Key benefits:
- Rich in potassium (good for blood pressure) and vitamin B6
- Vitamin C, dietary fiber, and some minerals
- Convenient, portable, and low in fat

Things to keep in mind:
- Contain natural sugars; portion matters if you’re watching carbs or blood sugar
- Ripe bananas are sweeter; greener ones have more resistant starch
- Low in protein and fat, so pair with a protein source for a balanced snack

Typical guidance: 1–2 bananas per day fits many diets, but adjust for your total carbohydrate goals and activity level.

_Good Job!_